In [1]:
!pip install transformers[sentencepiece]
!pip install emoji
!pip install datasets
!pip install tensorflow

     |████████████████████████████████| 3.8 MB 12.2 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 895 kB 38.8 MB/s 
     |████████████████████████████████| 6.5 MB 35.7 MB/s 
     |████████████████████████████████| 596 kB 44.1 MB/s 
     |████████████████████████████████| 1.2 MB 46.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 175 kB 11.5 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=8d40cce888391fa727782828cd7a0c49b92344f278125dba7ee6ad793938c167
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
     |████████████████████████████████| 312 kB 12.2 MB/s 
     |████████████████████████████████| 1.1 MB 47.9 MB/s 
     |████████████████████████████████| 134 kB 49.

In [2]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=2)


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_layer_norm', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [32]:
from transformers import AutoTokenizer, DataCollatorWithPadding

from datasets import load_dataset

imdb_dataset = load_dataset("imdb")
imdb_dataset=imdb_dataset['train'].shuffle().select(range(20000))

Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [33]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

imdb_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 20000
})

In [34]:
imdb_dataset[0]

{'label': 0,
 'text': 'Nahhh! Leila (Grace Mills) is a teenager turned on to Satan (and LSD) by her archaeologist fiancé Richard. There\'s a neighborhood hippie demon cult hanging out at the local decrepit ancient castle, where Leila and Richard drink blood, drop drugs, join in sex orgies, dance to lame psychedelic rock and participate in black mass ceremonies where the guys wear pants, masks and capes and the women don\'t wear anything at all. Unfortunately, these kind of extracurricular activities have left Leila open to demonic possession from the dreaded "spirit of evil." Leila also comes from a screwed up family, which doesn\'t help either. Her older brother John is a recluse who seems to be in love with her. Her mother Patricia (Maria Perschy) is depressed because she thinks she was responsible for the father\'s death. To top it off, her sister (Maria Kosti) is a slutty semi pro golfer named (gasp!) Debbie Gibson. There\'s plenty of hired help around also to waste more time. Two 

In [35]:
dataset_clean = imdb_dataset.train_test_split(train_size=0.8)

dataset_clean

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 4000
    })
})

In [36]:
tokenized_datasets = dataset_clean.map(tokenize_function, batched=True)

tokenized_datasets

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 16000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4000
    })
})

In [37]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)


In [38]:
import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 8
num_epochs = 3
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

In [39]:
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)


loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=1)

2000/2000 [==============================] - 1952s 971ms/step - loss: 0.2383 - accuracy: 0.9086 - val_loss: 0.1735 - val_accuracy: 0.9337


In [40]:
# !mkdir -p saved_model
# model.save('saved_model/my_model') 

In [41]:
# !zip -r /content/file.zip /content/saved_model

In [42]:
# from google.colab import files
# files.download("/content/file.zip")

In [43]:
sequences = ["It was not a good movie","The plot could have been better",'Really enjoyed the action scenes in the movie']
tokenized=tokenizer(sequences, padding=True, truncation=True, return_tensors="tf")['input_ids']
logit_s=model.predict(tokenized)['logits']
tf.argmax(tf.nn.softmax(logit_s),axis=1)

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([0, 0, 1])>